In [42]:
import pandas as pd
pd.set_option('display.max_columns', 50000)
import numpy as np
import csv
import json
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import normalize
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import datetime, os

In [43]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
import keras.backend as K
def get_f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

Num GPUs Available:  1


In [44]:
df = pd.read_csv("./dataset/train.csv")
df.head()

,customer_id,Name,age,gender,security_no,region_category,membership_category,joining_date,joined_through_referral,referral_id,preferred_offer_types,medium_of_operation,internet_option,last_visit_time,days_since_last_login,avg_time_spent,avg_transaction_value,avg_frequency_login_days,points_in_wallet,used_special_discount,offer_application_preference,past_complaint,complaint_status,feedback,churn_risk_score
0,fffe4300490044003600300030003800,Pattie Morrisey,18,F,XW0DQ7H,Village,Platinum Membership,2017-08-17,No,xxxxxxxx,Gift Vouchers/Coupons,?,Wi-Fi,16:08:02,17,300.63,53005.25,17.0,781.75,Yes,Yes,No,Not Applicable,Products always in Stock,2
1,fffe43004900440032003100300035003700,Traci Peery,32,F,5K0N3X1,City,Premium Membership,2017-08-28,?,CID21329,Gift Vouchers/Coupons,Desktop,Mobile_Data,12:38:13,16,306.34,12838.38,10.0,NaN,Yes,No,Yes,Solved,Quality Customer Care,1
2,fffe4300490044003100390032003600,Merideth Mcmeen,44,F,1F2TCL3,Town,No Membership,2016-11-11,Yes,CID12313,Gift Vouchers/Coupons,Desktop,Wi-Fi,22:53:21,14,516.16,21027.00,22.0,500.69,No,Yes,Yes,Solved in Follow-up,Poor Website,5
3,fffe43004900440036003000330031003600,Eufemia Cardwell,37,M,VJGJ33N,City,No Membership,2016-10-29,Yes,CID3793,Gift Vouchers/Coupons,Desktop,Mobile_Data,15:57:50,11,53.27,25239.56,6.0,567.66,No,Yes,Yes,Unsolved,Poor Website,5
4,fffe43004900440031003900350030003600,Meghan Kosak,31,F,SVZXCWB,City,No Membership,2017-09-12,No,xxxxxxxx,Credit/Debit Card Offers,Smartphone,Mobile_Data,15:46:44,20,113.13,24483.66,16.0,663.06,No,Yes,Yes,Solved,Poor Website,5


In [45]:
df.describe()

,age,days_since_last_login,avg_time_spent,avg_transaction_value,points_in_wallet,churn_risk_score
count,36992.000000,36992.000000,36992.000000,36992.000000,33549.000000,36992.000000
mean,37.118161,-41.915576,243.472334,29271.194003,686.882199,3.463397
std,15.867412,228.819900,398.289149,19444.806226,194.063624,1.409661
min,10.000000,-999.000000,-2814.109110,800.460000,-760.661236,-1.000000
25%,23.000000,8.000000,60.102500,14177.540000,616.150000,3.000000
50%,37.000000,12.000000,161.765000,27554.485000,697.620000,4.000000
75%,51.000000,16.000000,356.515000,40855.110000,763.950000,5.000000
max,64.000000,26.000000,3235.578521,99914.050000,2069.069761,5.000000


In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36992 entries, 0 to 36991
Data columns (total 25 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   customer_id                   36992 non-null  object 
 1   Name                          36992 non-null  object 
 2   age                           36992 non-null  int64  
 3   gender                        36992 non-null  object 
 4   security_no                   36992 non-null  object 
 5   region_category               31564 non-null  object 
 6   membership_category           36992 non-null  object 
 7   joining_date                  36992 non-null  object 
 8   joined_through_referral       36992 non-null  object 
 9   referral_id                   36992 non-null  object 
 10  preferred_offer_types         36704 non-null  object 
 11  medium_of_operation           36992 non-null  object 
 12  internet_option               36992 non-null  object 
 13  l

In [47]:
def valueCounts(df):
    print(df["gender"].value_counts())
    print("-"*50)
    print(df["region_category"].value_counts())
    print("-"*50)
    print(df["membership_category"].value_counts())
    print("-"*50)
    print(df["joined_through_referral"].value_counts())
    print("-"*50)
    print(df["preferred_offer_types"].value_counts())
    print("-"*50)
    print(df["medium_of_operation"].value_counts())
    print("-"*50)
    print(df["internet_option"].value_counts())
    print("-"*50)
    print(df["used_special_discount"].value_counts())
    print("-"*50)
    print(df["offer_application_preference"].value_counts())
    print("-"*50)
    print(df["past_complaint"].value_counts())
    print("-"*50)
    print(df["complaint_status"].value_counts())
    print("-"*50)
    print(df["feedback"].value_counts())
    print("-"*50)
    print(df["avg_frequency_login_days"].value_counts())
    print("-"*50)
    print(df["churn_risk_score"].value_counts())
    print("-"*50)
    
valueCounts(df)

F          18490
M          18443
Unknown       59
Name: gender, dtype: int64
--------------------------------------------------
Town       14128
City       12737
Village     4699
Name: region_category, dtype: int64
--------------------------------------------------
Basic Membership       7724
No Membership          7692
Gold Membership        6795
Silver Membership      5988
Premium Membership     4455
Platinum Membership    4338
Name: membership_category, dtype: int64
--------------------------------------------------
No     15839
Yes    15715
?       5438
Name: joined_through_referral, dtype: int64
--------------------------------------------------
Gift Vouchers/Coupons       12349
Credit/Debit Card Offers    12274
Without Offers              12081
Name: preferred_offer_types, dtype: int64
--------------------------------------------------
Desktop       13913
Smartphone    13876
?              5393
Both           3810
Name: medium_of_operation, dtype: int64
-------------------------

In [48]:
df.isnull().values.any()

True

In [49]:
df.isnull().sum()

customer_id                        0
Name                               0
age                                0
gender                             0
security_no                        0
region_category                 5428
membership_category                0
joining_date                       0
joined_through_referral            0
referral_id                        0
preferred_offer_types            288
medium_of_operation                0
internet_option                    0
last_visit_time                    0
days_since_last_login              0
avg_time_spent                     0
avg_transaction_value              0
avg_frequency_login_days           0
points_in_wallet                3443
used_special_discount              0
offer_application_preference       0
past_complaint                     0
complaint_status                   0
feedback                           0
churn_risk_score                   0
dtype: int64

In [50]:
# setting target wrong value -1 to 1 assuming sign issue, 
df['churn_risk_score'] = df['churn_risk_score'].apply(lambda x:1 if x == -1 else x)
df['churn_risk_score'] = df['churn_risk_score'].apply(lambda x:x-1)

In [51]:
def handleMissingValues(df):
    
    df['joining_date']  = pd.to_datetime(df['joining_date'],format="%Y-%m-%d")
    
    
    # replacing Unknown words with most occured in df for gender column
    df['gender'] = df['gender'].replace(['Unknown'], df["gender"].mode()[0])
    df['joined_through_referral'] = df['joined_through_referral'].replace(['?'], df["joined_through_referral"].mode()[0])
    df['avg_frequency_login_days'] = df['avg_frequency_login_days'].replace(['Error'], None)
    
    
    df["region_category"] = df["region_category"].fillna(df["region_category"].mode()[0])
    df["preferred_offer_types"] = df["preferred_offer_types"].fillna(df["preferred_offer_types"].mode()[0])
    df["points_in_wallet"] = df["points_in_wallet"].fillna(df["points_in_wallet"].median())
    df["avg_frequency_login_days"] = df["avg_frequency_login_days"].fillna(df["avg_frequency_login_days"].median())
    
    
    df["avg_frequency_login_days"] = pd.to_numeric(df["avg_frequency_login_days"])
    
    return df

df = handleMissingValues(df)
df.isnull().sum()

customer_id                     0
Name                            0
age                             0
gender                          0
security_no                     0
region_category                 0
membership_category             0
joining_date                    0
joined_through_referral         0
referral_id                     0
preferred_offer_types           0
medium_of_operation             0
internet_option                 0
last_visit_time                 0
days_since_last_login           0
avg_time_spent                  0
avg_transaction_value           0
avg_frequency_login_days        0
points_in_wallet                0
used_special_discount           0
offer_application_preference    0
past_complaint                  0
complaint_status                0
feedback                        0
churn_risk_score                0
dtype: int64

In [52]:
valueCounts(df)

F    18549
M    18443
Name: gender, dtype: int64
--------------------------------------------------
Town       19556
City       12737
Village     4699
Name: region_category, dtype: int64
--------------------------------------------------
Basic Membership       7724
No Membership          7692
Gold Membership        6795
Silver Membership      5988
Premium Membership     4455
Platinum Membership    4338
Name: membership_category, dtype: int64
--------------------------------------------------
No     21277
Yes    15715
Name: joined_through_referral, dtype: int64
--------------------------------------------------
Gift Vouchers/Coupons       12637
Credit/Debit Card Offers    12274
Without Offers              12081
Name: preferred_offer_types, dtype: int64
--------------------------------------------------
Desktop       13913
Smartphone    13876
?              5393
Both           3810
Name: medium_of_operation, dtype: int64
--------------------------------------------------
Wi-Fi          1

In [53]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36992 entries, 0 to 36991
Data columns (total 25 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   customer_id                   36992 non-null  object        
 1   Name                          36992 non-null  object        
 2   age                           36992 non-null  int64         
 3   gender                        36992 non-null  object        
 4   security_no                   36992 non-null  object        
 5   region_category               36992 non-null  object        
 6   membership_category           36992 non-null  object        
 7   joining_date                  36992 non-null  datetime64[ns]
 8   joined_through_referral       36992 non-null  object        
 9   referral_id                   36992 non-null  object        
 10  preferred_offer_types         36992 non-null  object        
 11  medium_of_operation         

In [54]:
def handleExtraFeatures(df):

    from datetime import date
    df['todays_date'] = date.today().strftime("%Y-%m-%d")
    df['todays_date']  = pd.to_datetime(df['todays_date'],format="%Y-%m-%d")
    

    df['customer_days'] = df['todays_date'] - df['joining_date'] 
    df['customer_days'] = df['customer_days'] / np.timedelta64(1,'D')

    return df
df = handleExtraFeatures(df)
df.head()

,customer_id,Name,age,gender,security_no,region_category,membership_category,joining_date,joined_through_referral,referral_id,preferred_offer_types,medium_of_operation,internet_option,last_visit_time,days_since_last_login,avg_time_spent,avg_transaction_value,avg_frequency_login_days,points_in_wallet,used_special_discount,offer_application_preference,past_complaint,complaint_status,feedback,churn_risk_score,todays_date,customer_days
0,fffe4300490044003600300030003800,Pattie Morrisey,18,F,XW0DQ7H,Village,Platinum Membership,2017-08-17,No,xxxxxxxx,Gift Vouchers/Coupons,?,Wi-Fi,16:08:02,17,300.63,53005.25,17.0,781.75,Yes,Yes,No,Not Applicable,Products always in Stock,1,2021-03-27,1318.0
1,fffe43004900440032003100300035003700,Traci Peery,32,F,5K0N3X1,City,Premium Membership,2017-08-28,No,CID21329,Gift Vouchers/Coupons,Desktop,Mobile_Data,12:38:13,16,306.34,12838.38,10.0,697.62,Yes,No,Yes,Solved,Quality Customer Care,0,2021-03-27,1307.0
2,fffe4300490044003100390032003600,Merideth Mcmeen,44,F,1F2TCL3,Town,No Membership,2016-11-11,Yes,CID12313,Gift Vouchers/Coupons,Desktop,Wi-Fi,22:53:21,14,516.16,21027.00,22.0,500.69,No,Yes,Yes,Solved in Follow-up,Poor Website,4,2021-03-27,1597.0
3,fffe43004900440036003000330031003600,Eufemia Cardwell,37,M,VJGJ33N,City,No Membership,2016-10-29,Yes,CID3793,Gift Vouchers/Coupons,Desktop,Mobile_Data,15:57:50,11,53.27,25239.56,6.0,567.66,No,Yes,Yes,Unsolved,Poor Website,4,2021-03-27,1610.0
4,fffe43004900440031003900350030003600,Meghan Kosak,31,F,SVZXCWB,City,No Membership,2017-09-12,No,xxxxxxxx,Credit/Debit Card Offers,Smartphone,Mobile_Data,15:46:44,20,113.13,24483.66,16.0,663.06,No,Yes,Yes,Solved,Poor Website,4,2021-03-27,1292.0


In [55]:
def labelEncoding(df, onlyTransform: False, label_object = {}):
    categorical_columns = [
        'gender',
        'joined_through_referral',
        'used_special_discount',
        'offer_application_preference',
        'past_complaint'
    ]
    for col in categorical_columns:
        if col in df.columns:
            labelencoder = None
            if onlyTransform is False:
                labelencoder = LabelEncoder()
                labelencoder.fit(df[col])
                label_object[col] = labelencoder
            else:
                labelencoder = label_object[col]

            df[col] = labelencoder.transform(df[col])
    return df, label_object

df, label_object = labelEncoding(df, onlyTransform= False, label_object= {})
df.head()

,customer_id,Name,age,gender,security_no,region_category,membership_category,joining_date,joined_through_referral,referral_id,preferred_offer_types,medium_of_operation,internet_option,last_visit_time,days_since_last_login,avg_time_spent,avg_transaction_value,avg_frequency_login_days,points_in_wallet,used_special_discount,offer_application_preference,past_complaint,complaint_status,feedback,churn_risk_score,todays_date,customer_days
0,fffe4300490044003600300030003800,Pattie Morrisey,18,0,XW0DQ7H,Village,Platinum Membership,2017-08-17,0,xxxxxxxx,Gift Vouchers/Coupons,?,Wi-Fi,16:08:02,17,300.63,53005.25,17.0,781.75,1,1,0,Not Applicable,Products always in Stock,1,2021-03-27,1318.0
1,fffe43004900440032003100300035003700,Traci Peery,32,0,5K0N3X1,City,Premium Membership,2017-08-28,0,CID21329,Gift Vouchers/Coupons,Desktop,Mobile_Data,12:38:13,16,306.34,12838.38,10.0,697.62,1,0,1,Solved,Quality Customer Care,0,2021-03-27,1307.0
2,fffe4300490044003100390032003600,Merideth Mcmeen,44,0,1F2TCL3,Town,No Membership,2016-11-11,1,CID12313,Gift Vouchers/Coupons,Desktop,Wi-Fi,22:53:21,14,516.16,21027.00,22.0,500.69,0,1,1,Solved in Follow-up,Poor Website,4,2021-03-27,1597.0
3,fffe43004900440036003000330031003600,Eufemia Cardwell,37,1,VJGJ33N,City,No Membership,2016-10-29,1,CID3793,Gift Vouchers/Coupons,Desktop,Mobile_Data,15:57:50,11,53.27,25239.56,6.0,567.66,0,1,1,Unsolved,Poor Website,4,2021-03-27,1610.0
4,fffe43004900440031003900350030003600,Meghan Kosak,31,0,SVZXCWB,City,No Membership,2017-09-12,0,xxxxxxxx,Credit/Debit Card Offers,Smartphone,Mobile_Data,15:46:44,20,113.13,24483.66,16.0,663.06,0,1,1,Solved,Poor Website,4,2021-03-27,1292.0


In [56]:
def encodeMembership(df):
    
    cleanup_nums = {
                    "membership_category":{
                        "No Membership":0,
                        "Basic Membership":1,
                        "Gold Membership":2,
                        "Silver Membership":3,
                        "Premium Membership":4,
                        "Platinum Membership":5
                    }
                   }
    
    df = df.replace(cleanup_nums)
    
    return df

df = encodeMembership(df)
df.head()

,customer_id,Name,age,gender,security_no,region_category,membership_category,joining_date,joined_through_referral,referral_id,preferred_offer_types,medium_of_operation,internet_option,last_visit_time,days_since_last_login,avg_time_spent,avg_transaction_value,avg_frequency_login_days,points_in_wallet,used_special_discount,offer_application_preference,past_complaint,complaint_status,feedback,churn_risk_score,todays_date,customer_days
0,fffe4300490044003600300030003800,Pattie Morrisey,18,0,XW0DQ7H,Village,5,2017-08-17,0,xxxxxxxx,Gift Vouchers/Coupons,?,Wi-Fi,16:08:02,17,300.63,53005.25,17.0,781.75,1,1,0,Not Applicable,Products always in Stock,1,2021-03-27,1318.0
1,fffe43004900440032003100300035003700,Traci Peery,32,0,5K0N3X1,City,4,2017-08-28,0,CID21329,Gift Vouchers/Coupons,Desktop,Mobile_Data,12:38:13,16,306.34,12838.38,10.0,697.62,1,0,1,Solved,Quality Customer Care,0,2021-03-27,1307.0
2,fffe4300490044003100390032003600,Merideth Mcmeen,44,0,1F2TCL3,Town,0,2016-11-11,1,CID12313,Gift Vouchers/Coupons,Desktop,Wi-Fi,22:53:21,14,516.16,21027.00,22.0,500.69,0,1,1,Solved in Follow-up,Poor Website,4,2021-03-27,1597.0
3,fffe43004900440036003000330031003600,Eufemia Cardwell,37,1,VJGJ33N,City,0,2016-10-29,1,CID3793,Gift Vouchers/Coupons,Desktop,Mobile_Data,15:57:50,11,53.27,25239.56,6.0,567.66,0,1,1,Unsolved,Poor Website,4,2021-03-27,1610.0
4,fffe43004900440031003900350030003600,Meghan Kosak,31,0,SVZXCWB,City,0,2017-09-12,0,xxxxxxxx,Credit/Debit Card Offers,Smartphone,Mobile_Data,15:46:44,20,113.13,24483.66,16.0,663.06,0,1,1,Solved,Poor Website,4,2021-03-27,1292.0


In [57]:
def oneHotEncoding(df, onlyTransform=False, one_hot_object = {}):
    categorical_columns = [
        'region_category',
        'preferred_offer_types',
        'medium_of_operation',
        'internet_option',
        'complaint_status',
        'feedback'
    ]
    for col in categorical_columns:
        
        col_data  = df[col].values.reshape(-1,1)
        
        if onlyTransform is False:
            onehot_encoder = OneHotEncoder()
            onehot_encoder = onehot_encoder.fit(col_data)
            one_hot_object[col] = onehot_encoder
        
        onehot_encoded = one_hot_object[col].transform(col_data).toarray()
        columns_list = []
        for i in range(onehot_encoded.shape[1]):
            columns_list.append(col+"_"+str(int(i)))
        dfOneHot = pd.DataFrame(onehot_encoded, columns = columns_list) 
        df = pd.concat([df, dfOneHot], axis=1)
        

    df = df.drop(categorical_columns,axis=1)
    return df, one_hot_object


df, one_hot_object = oneHotEncoding(df, onlyTransform=False, one_hot_object = {})
df.head()

,customer_id,Name,age,gender,security_no,membership_category,joining_date,joined_through_referral,referral_id,last_visit_time,days_since_last_login,avg_time_spent,avg_transaction_value,avg_frequency_login_days,points_in_wallet,used_special_discount,offer_application_preference,past_complaint,churn_risk_score,todays_date,customer_days,region_category_0,region_category_1,region_category_2,preferred_offer_types_0,preferred_offer_types_1,preferred_offer_types_2,medium_of_operation_0,medium_of_operation_1,medium_of_operation_2,medium_of_operation_3,internet_option_0,internet_option_1,internet_option_2,complaint_status_0,complaint_status_1,complaint_status_2,complaint_status_3,complaint_status_4,feedback_0,feedback_1,feedback_2,feedback_3,feedback_4,feedback_5,feedback_6,feedback_7,feedback_8
0,fffe4300490044003600300030003800,Pattie Morrisey,18,0,XW0DQ7H,5,2017-08-17,0,xxxxxxxx,16:08:02,17,300.63,53005.25,17.0,781.75,1,1,0,1,2021-03-27,1318.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,fffe43004900440032003100300035003700,Traci Peery,32,0,5K0N3X1,4,2017-08-28,0,CID21329,12:38:13,16,306.34,12838.38,10.0,697.62,1,0,1,0,2021-03-27,1307.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,fffe4300490044003100390032003600,Merideth Mcmeen,44,0,1F2TCL3,0,2016-11-11,1,CID12313,22:53:21,14,516.16,21027.00,22.0,500.69,0,1,1,4,2021-03-27,1597.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,fffe43004900440036003000330031003600,Eufemia Cardwell,37,1,VJGJ33N,0,2016-10-29,1,CID3793,15:57:50,11,53.27,25239.56,6.0,567.66,0,1,1,4,2021-03-27,1610.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,fffe43004900440031003900350030003600,Meghan Kosak,31,0,SVZXCWB,0,2017-09-12,0,xxxxxxxx,15:46:44,20,113.13,24483.66,16.0,663.06,0,1,1,4,2021-03-27,1292.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [58]:
def removeRedundantColumns(df):
    
    redundant_columns = ['customer_id','Name','security_no','joining_date','referral_id','last_visit_time','todays_date']

    df = df.drop(redundant_columns,axis=1)
    
    return df

df = removeRedundantColumns(df)
df.head()

,age,gender,membership_category,joined_through_referral,days_since_last_login,avg_time_spent,avg_transaction_value,avg_frequency_login_days,points_in_wallet,used_special_discount,offer_application_preference,past_complaint,churn_risk_score,customer_days,region_category_0,region_category_1,region_category_2,preferred_offer_types_0,preferred_offer_types_1,preferred_offer_types_2,medium_of_operation_0,medium_of_operation_1,medium_of_operation_2,medium_of_operation_3,internet_option_0,internet_option_1,internet_option_2,complaint_status_0,complaint_status_1,complaint_status_2,complaint_status_3,complaint_status_4,feedback_0,feedback_1,feedback_2,feedback_3,feedback_4,feedback_5,feedback_6,feedback_7,feedback_8
0,18,0,5,0,17,300.63,53005.25,17.0,781.75,1,1,0,1,1318.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,32,0,4,0,16,306.34,12838.38,10.0,697.62,1,0,1,0,1307.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,44,0,0,1,14,516.16,21027.00,22.0,500.69,0,1,1,4,1597.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,37,1,0,1,11,53.27,25239.56,6.0,567.66,0,1,1,4,1610.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,31,0,0,0,20,113.13,24483.66,16.0,663.06,0,1,1,4,1292.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [59]:
from keras.utils import np_utils
y = pd.DataFrame(df["churn_risk_score"])
y = np_utils.to_categorical(df["churn_risk_score"])
df = df.drop("churn_risk_score", axis=1)

X = pd.DataFrame(df)
print(X.shape)
print(y.shape)

(36992, 40)
(36992, 5)


In [60]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, stratify=y)

X_train = X
y_train = y
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(36992, 40)
(36992, 5)
(11098, 40)
(11098, 5)


In [61]:

from sklearn.preprocessing import StandardScaler, MinMaxScaler

scalerX = StandardScaler()
scalerX = scalerX.fit(X_train)
X_train = scalerX.transform(X_train)
X_train = pd.DataFrame(X_train)

X_test = scalerX.transform(X_test)
X_test = pd.DataFrame(X_test)

In [62]:
X_train

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39
0,-1.204886,-0.997139,1.739411,-0.859413,0.257479,0.143510,1.220602,0.109346,0.507848,0.904712,0.899880,-0.994285,-1.286093,-0.724658,-1.059050,2.62151,-0.704671,1.388264,-0.696395,2.420591,-0.338853,-0.776429,-0.774775,-0.703039,-0.707637,1.407161,-0.374500,0.994285,-0.377731,-0.375766,-0.378898,-0.452629,-0.450980,-0.455227,-0.451805,5.076121,-0.195366,-0.199578,-0.452152,-0.197666
1,-0.322562,-0.997139,1.133373,-0.859413,0.253109,0.157846,-0.845112,-0.651811,0.052687,0.904712,-1.111259,1.005747,-1.320699,1.379962,-1.059050,-0.38146,-0.704671,1.388264,-0.696395,-0.413122,-0.338853,1.287947,-0.774775,-0.703039,1.413153,-0.710651,-0.374500,-1.005747,2.647387,-0.375766,-0.378898,-0.452629,-0.450980,-0.455227,-0.451805,-0.197001,5.118594,-0.199578,-0.452152,-0.197666
2,0.433715,-0.997139,-1.290780,1.163585,0.244368,0.684657,-0.423985,0.653030,-1.012747,-1.105324,0.899880,1.005747,-0.408338,-0.724658,0.944242,-0.38146,-0.704671,1.388264,-0.696395,-0.413122,-0.338853,1.287947,-0.774775,-0.703039,-0.707637,1.407161,-0.374500,-1.005747,-0.377731,2.661231,-0.378898,-0.452629,-0.450980,-0.455227,2.213346,-0.197001,-0.195366,-0.199578,-0.452152,-0.197666
3,-0.007447,1.002870,-1.290780,1.163585,0.231257,-0.477555,-0.207340,-1.086758,-0.650425,-1.105324,0.899880,1.005747,-0.367439,1.379962,-1.059050,-0.38146,-0.704671,1.388264,-0.696395,-0.413122,-0.338853,1.287947,-0.774775,-0.703039,1.413153,-0.710651,-0.374500,-1.005747,-0.377731,-0.375766,2.639232,-0.452629,-0.450980,-0.455227,2.213346,-0.197001,-0.195366,-0.199578,-0.452152,-0.197666
4,-0.385585,-0.997139,-1.290780,-0.859413,0.270590,-0.327260,-0.246215,0.000609,-0.134290,-1.105324,0.899880,1.005747,-1.367890,1.379962,-1.059050,-0.38146,1.419102,-0.720324,-0.696395,-0.413122,-0.338853,-0.776429,1.290697,-0.703039,1.413153,-0.710651,-0.374500,-1.005747,2.647387,-0.375766,-0.378898,-0.452629,-0.450980,-0.455227,2.213346,-0.197001,-0.195366,-0.199578,-0.452152,-0.197666
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36987,0.559761,-0.997139,-0.684742,-0.859413,0.191925,-2.245020,-0.102523,-1.086758,-0.261701,-1.105324,0.899880,1.005747,-1.396205,-0.724658,0.944242,-0.38146,1.419102,-0.720324,-0.696395,-0.413122,-0.338853,1.287947,-0.774775,-0.703039,-0.707637,1.407161,2.670227,-1.005747,-0.377731,-0.375766,-0.378898,2.209317,-0.450980,-0.455227,-0.451805,-0.197001,-0.195366,-0.199578,-0.452152,-0.197666
36988,-0.511632,-0.997139,-0.684742,-0.859413,0.239998,-2.213487,-0.936072,1.305450,-0.865048,0.904712,-1.111259,-0.994285,0.022674,-0.724658,0.944242,-0.38146,-0.704671,-0.720324,1.435966,-0.413122,-0.338853,-0.776429,1.290697,-0.703039,-0.707637,1.407161,-0.374500,0.994285,-0.377731,-0.375766,-0.378898,-0.452629,2.217392,-0.455227,-0.451805,-0.197001,-0.195366,-0.199578,-0.452152,-0.197666
36989,-0.889770,-0.997139,-0.684742,1.163585,0.235628,-0.222285,0.455468,1.305450,-0.040098,-1.105324,0.899880,1.005747,-0.216428,-0.724658,0.944242,-0.38146,-0.704671,1.388264,-0.696395,-0.413122,-0.338853,1.287947,-0.774775,-0.703039,-0.707637,1.407161,-0.374500,-1.005747,-0.377731,-0.375766,2.639232,-0.452629,-0.450980,-0.455227,2.213346,-0.197001,-0.195366,-0.199578,-0.452152,-0.197666
36990,1.000923,1.002870,1.739411,-0.859413,0.248739,0.600420,-1.383027,0.435556,-2.654345,0.904712,0.899880,-0.994285,-1.087890,-0.724658,-1.059050,2.62151,-0.704671,1.388264,-0.696395,-0.413122,-0.338853,-0.776429,1.290697,-0.703039,1.413153,-0.710651,-0.374500,0.994285,-0.377731,-0.375766,-0.378898,2.209317,-0.450980,-0.455227,-0.451805,-0.197001,-0.195366,-0.199578,-0.452152,-0.197666


In [63]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Activation,Dropout 
from tensorflow.keras.regularizers import L1L2
from tensorflow.keras.optimizers import Adam

model = Sequential()
model.add(Dense(1024,input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(y_train.shape[1], activation='softmax'))
model.compile(optimizer='adam',
                loss='categorical_crossentropy',
                metrics=['accuracy',get_f1])

print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 1024)              41984     
_________________________________________________________________
dense_6 (Dense)              (None, 512)               524800    
_________________________________________________________________
dense_7 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_8 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_9 (Dense)              (None, 5)                 645       
Total params: 731,653
Trainable params: 731,653
Non-trainable params: 0
_________________________________________________________________
None


In [64]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),batch_size=1000,epochs=50,verbose=1)

Epoch 1/50
37/37 [==============================] - 2s 23ms/step - loss: 1.0615 - accuracy: 0.5194 - get_f1: 0.3539 - val_loss: 0.7536 - val_accuracy: 0.6325 - val_get_f1: 0.6209
Epoch 2/50
37/37 [==============================] - 0s 13ms/step - loss: 0.7282 - accuracy: 0.6290 - get_f1: 0.6154 - val_loss: 0.6897 - val_accuracy: 0.6545 - val_get_f1: 0.6526
Epoch 3/50
37/37 [==============================] - 1s 14ms/step - loss: 0.6826 - accuracy: 0.6556 - get_f1: 0.6472 - val_loss: 0.6543 - val_accuracy: 0.6641 - val_get_f1: 0.6604
Epoch 4/50
37/37 [==============================] - 0s 13ms/step - loss: 0.6521 - accuracy: 0.6832 - get_f1: 0.6764 - val_loss: 0.6245 - val_accuracy: 0.7117 - val_get_f1: 0.7014
Epoch 5/50
37/37 [==============================] - 0s 13ms/step - loss: 0.6173 - accuracy: 0.7080 - get_f1: 0.7036 - val_loss: 0.5899 - val_accuracy: 0.7246 - val_get_f1: 0.7193
Epoch 6/50
37/37 [==============================] - 0s 13ms/step - loss: 0.5792 - accuracy: 0.7274 - get_

37/37 [==============================] - 1s 14ms/step - loss: 0.0312 - accuracy: 0.9892 - get_f1: 0.9892 - val_loss: 0.0398 - val_accuracy: 0.9866 - val_get_f1: 0.9868
Epoch 47/50
37/37 [==============================] - 0s 13ms/step - loss: 0.0564 - accuracy: 0.9797 - get_f1: 0.9797 - val_loss: 0.0715 - val_accuracy: 0.9735 - val_get_f1: 0.9725
Epoch 48/50
37/37 [==============================] - 0s 13ms/step - loss: 0.0860 - accuracy: 0.9682 - get_f1: 0.9682 - val_loss: 0.0662 - val_accuracy: 0.9756 - val_get_f1: 0.9743
Epoch 49/50
37/37 [==============================] - 0s 13ms/step - loss: 0.0872 - accuracy: 0.9672 - get_f1: 0.9672 - val_loss: 0.0871 - val_accuracy: 0.9662 - val_get_f1: 0.9658
Epoch 50/50
37/37 [==============================] - 0s 13ms/step - loss: 0.0919 - accuracy: 0.9645 - get_f1: 0.9646 - val_loss: 0.0531 - val_accuracy: 0.9814 - val_get_f1: 0.9798


In [65]:
prediction = model.predict(X_test)

In [66]:
prediction

array([[3.0612625e-06, 2.1011548e-20, 9.9999690e-01, 3.2478013e-17,
        0.0000000e+00],
       [2.4527091e-09, 4.6067777e-19, 1.0000000e+00, 3.2058331e-08,
        5.8199818e-34],
       [4.4548317e-04, 1.3413427e-13, 1.1428738e-19, 5.2112659e-06,
        9.9954933e-01],
       ...,
       [2.2298060e-07, 2.3072546e-12, 1.4543495e-11, 8.4341788e-01,
        1.5658191e-01],
       [9.9068820e-01, 9.3117375e-03, 7.1193221e-15, 1.9623725e-08,
        1.8751160e-10],
       [2.4295599e-07, 5.4196665e-11, 1.9563326e-05, 9.9998021e-01,
        4.9734704e-22]], dtype=float32)

In [67]:
y_label = tf.math.argmax(y_test, axis=1)
predict_label = tf.math.argmax(prediction,axis=1)

print(y_label)
print(predict_label)
# print(label_object['churn_risk_score'].inverse_transform(predict_label))

tf.Tensor([2 2 4 ... 3 0 3], shape=(11098,), dtype=int64)
tf.Tensor([2 2 4 ... 3 0 3], shape=(11098,), dtype=int64)


In [68]:
from sklearn.metrics import accuracy_score as sk_accuracy_score
from sklearn.metrics import f1_score as sk_f1_score
from sklearn.metrics import precision_score as sk_precision_score
from sklearn.metrics import recall_score as sk_recall_score
from sklearn.metrics import confusion_matrix as sk_confusion_matrix

print("sk_accuracy_score :",100*sk_accuracy_score(y_label, predict_label))
print("sk_f1_score :",100*sk_f1_score(y_label, predict_label,average="macro"))
print("sk_precision_score :", 100*sk_precision_score(y_label, predict_label,average="macro"))
print("sk_recall_score :" ,100*sk_recall_score(y_label, predict_label,average="macro"))

sk_accuracy_score : 98.14380969544062
sk_f1_score : 98.23983675647983
sk_precision_score : 98.2551848134159
sk_recall_score : 98.23278411504782


In [69]:
df_test = pd.read_csv("./dataset/test.csv")
df_test.head()

,customer_id,Name,age,gender,security_no,region_category,membership_category,joining_date,joined_through_referral,referral_id,preferred_offer_types,medium_of_operation,internet_option,last_visit_time,days_since_last_login,avg_time_spent,avg_transaction_value,avg_frequency_login_days,points_in_wallet,used_special_discount,offer_application_preference,past_complaint,complaint_status,feedback
0,fffe43004900440031003700300030003400,Alethia Meints,50,F,OQJ1XAY,Village,Premium Membership,2015-11-02,No,xxxxxxxx,Without Offers,Smartphone,Wi-Fi,07:19:30,12,386.26,40721.44,7.0,733.830000,Yes,No,No,Not Applicable,Poor Product Quality
1,fffe43004900440031003900370037003300,Ming Lopez,41,M,OUQRPKO,Village,Gold Membership,2016-03-01,No,xxxxxxxx,Without Offers,Desktop,Fiber_Optic,22:21:16,11,37.80,9644.40,9.0,726.000000,Yes,No,No,Not Applicable,Poor Website
2,fffe43004900440034003800360037003000,Carina Flannigan,31,F,02J2RE7,Town,Silver Membership,2017-03-03,No,xxxxxxxx,Gift Vouchers/Coupons,Both,Mobile_Data,16:40:39,18,215.36,3693.25,21.0,713.780000,Yes,No,Yes,Solved in Follow-up,No reason specified
3,fffe43004900440036003200370033003400,Kyung Wanner,64,M,5YEQIF1,Town,Silver Membership,2017-08-18,Yes,CID8941,Credit/Debit Card Offers,?,Fiber_Optic,14:56:17,-999,44.57,36809.56,11.0,744.970000,Yes,No,Yes,No Information Available,Too many ads
4,fffe43004900440035003000370031003900,Enola Gatto,16,F,100RYB5,Town,No Membership,2015-05-05,Yes,CID5690,Without Offers,Smartphone,Mobile_Data,02:57:53,6,349.88,40675.86,8.0,299.048351,No,Yes,Yes,Solved in Follow-up,Poor Website


In [70]:
df_test =  handleMissingValues(df_test)

In [71]:
df_test = handleExtraFeatures(df_test)

In [72]:
df_test, label_object = labelEncoding(df_test, onlyTransform= True, label_object= label_object)

In [73]:
df_test = encodeMembership(df_test)

In [74]:
df_test, one_hot_object = oneHotEncoding(df_test, onlyTransform=True, one_hot_object = one_hot_object)

In [75]:
df_test.head()

,customer_id,Name,age,gender,security_no,membership_category,joining_date,joined_through_referral,referral_id,last_visit_time,days_since_last_login,avg_time_spent,avg_transaction_value,avg_frequency_login_days,points_in_wallet,used_special_discount,offer_application_preference,past_complaint,todays_date,customer_days,region_category_0,region_category_1,region_category_2,preferred_offer_types_0,preferred_offer_types_1,preferred_offer_types_2,medium_of_operation_0,medium_of_operation_1,medium_of_operation_2,medium_of_operation_3,internet_option_0,internet_option_1,internet_option_2,complaint_status_0,complaint_status_1,complaint_status_2,complaint_status_3,complaint_status_4,feedback_0,feedback_1,feedback_2,feedback_3,feedback_4,feedback_5,feedback_6,feedback_7,feedback_8
0,fffe43004900440031003700300030003400,Alethia Meints,50,0,OQJ1XAY,4,2015-11-02,0,xxxxxxxx,07:19:30,12,386.26,40721.44,7.0,733.830000,1,0,0,2021-03-27,1972.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,fffe43004900440031003900370037003300,Ming Lopez,41,1,OUQRPKO,2,2016-03-01,0,xxxxxxxx,22:21:16,11,37.80,9644.40,9.0,726.000000,1,0,0,2021-03-27,1852.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,fffe43004900440034003800360037003000,Carina Flannigan,31,0,02J2RE7,3,2017-03-03,0,xxxxxxxx,16:40:39,18,215.36,3693.25,21.0,713.780000,1,0,1,2021-03-27,1485.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,fffe43004900440036003200370033003400,Kyung Wanner,64,1,5YEQIF1,3,2017-08-18,1,CID8941,14:56:17,-999,44.57,36809.56,11.0,744.970000,1,0,1,2021-03-27,1317.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,fffe43004900440035003000370031003900,Enola Gatto,16,0,100RYB5,0,2015-05-05,1,CID5690,02:57:53,6,349.88,40675.86,8.0,299.048351,0,1,1,2021-03-27,2153.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [76]:
customer_ids = df_test["customer_id"]
customer_ids_df = pd.DataFrame(data = customer_ids,columns=["customer_id"])
df_test = removeRedundantColumns(df_test)
df_test.head()

,age,gender,membership_category,joined_through_referral,days_since_last_login,avg_time_spent,avg_transaction_value,avg_frequency_login_days,points_in_wallet,used_special_discount,offer_application_preference,past_complaint,customer_days,region_category_0,region_category_1,region_category_2,preferred_offer_types_0,preferred_offer_types_1,preferred_offer_types_2,medium_of_operation_0,medium_of_operation_1,medium_of_operation_2,medium_of_operation_3,internet_option_0,internet_option_1,internet_option_2,complaint_status_0,complaint_status_1,complaint_status_2,complaint_status_3,complaint_status_4,feedback_0,feedback_1,feedback_2,feedback_3,feedback_4,feedback_5,feedback_6,feedback_7,feedback_8
0,50,0,4,0,12,386.26,40721.44,7.0,733.830000,1,0,0,1972.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,41,1,2,0,11,37.80,9644.40,9.0,726.000000,1,0,0,1852.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,31,0,3,0,18,215.36,3693.25,21.0,713.780000,1,0,1,1485.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,64,1,3,1,-999,44.57,36809.56,11.0,744.970000,1,0,1,1317.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,16,0,0,1,6,349.88,40675.86,8.0,299.048351,0,1,1,2153.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [77]:
x_test_submit = scalerX.transform(df_test)
x_test_submit = pd.DataFrame(x_test_submit)
x_test_submit.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39
0,0.811853,-0.997139,1.133373,-0.859413,0.235628,0.358507,0.588867,-0.978021,0.248591,0.904712,-1.111259,-0.994285,0.771439,-0.724658,-1.059050,2.62151,-0.704671,-0.720324,1.435966,-0.413122,-0.338853,-0.776429,1.290697,-0.703039,-0.707637,1.407161,-0.374500,0.994285,-0.377731,-0.375766,-0.378898,-0.452629,-0.45098,2.196705,-0.451805,-0.197001,-0.195366,-0.199578,-0.452152,-0.197666
1,0.244646,1.002870,-0.078704,-0.859413,0.231257,-0.516396,-1.009373,-0.760548,0.206229,0.904712,-1.111259,-0.994285,0.393910,-0.724658,-1.059050,2.62151,-0.704671,-0.720324,1.435966,-0.413122,-0.338853,1.287947,-0.774775,1.422396,-0.707637,-0.710651,-0.374500,0.994285,-0.377731,-0.375766,-0.378898,-0.452629,-0.45098,-0.455227,2.213346,-0.197001,-0.195366,-0.199578,-0.452152,-0.197666
2,-0.385585,-0.997139,0.527335,-0.859413,0.261850,-0.070584,-1.315430,0.544293,0.140116,0.904712,-1.111259,1.005747,-0.760699,-0.724658,0.944242,-0.38146,-0.704671,1.388264,-0.696395,-0.413122,2.951133,-0.776429,-0.774775,-0.703039,1.413153,-0.710651,-0.374500,-1.005747,-0.377731,2.661231,-0.378898,2.209317,-0.45098,-0.455227,-0.451805,-0.197001,-0.195366,-0.199578,-0.452152,-0.197666
3,1.694177,1.002870,0.527335,1.163585,-4.182754,-0.499399,0.387685,-0.543074,0.308861,0.904712,-1.111259,1.005747,-1.289239,-0.724658,0.944242,-0.38146,1.419102,-0.720324,-0.696395,2.420591,-0.338853,-0.776429,-0.774775,1.422396,-0.707637,-0.710651,2.670227,-1.005747,-0.377731,-0.375766,-0.378898,-0.452629,-0.45098,-0.455227,-0.451805,-0.197001,-0.195366,-0.199578,2.211647,-0.197666
4,-1.330932,-0.997139,-1.290780,1.163585,0.209406,0.267165,0.586523,-0.869285,-2.103672,-1.105324,0.899880,1.005747,1.340878,-0.724658,0.944242,-0.38146,-0.704671,-0.720324,1.435966,-0.413122,-0.338853,-0.776429,1.290697,-0.703039,1.413153,-0.710651,-0.374500,-1.005747,-0.377731,2.661231,-0.378898,-0.452629,-0.45098,-0.455227,2.213346,-0.197001,-0.195366,-0.199578,-0.452152,-0.197666


In [78]:
prediction_submit = model.predict(x_test_submit)
predict_label = tf.math.argmax(prediction_submit,axis=1)

# predict_inversed = label_object['churn_risk_score'].inverse_transform(predict_label)
print(predict_label)
# print(predict_inversed)

tf.Tensor([2 2 2 ... 4 3 2], shape=(19919,), dtype=int64)


In [79]:
df_to_submit = pd.concat([customer_ids_df,pd.DataFrame(data = predict_label,columns=["churn_risk_score"])], axis=1)
df_to_submit['churn_risk_score'] = df_to_submit['churn_risk_score'].apply(lambda x:x+1)

In [80]:
df_to_submit.head()

,customer_id,churn_risk_score
0,fffe43004900440031003700300030003400,3
1,fffe43004900440031003900370037003300,3
2,fffe43004900440034003800360037003000,3
3,fffe43004900440036003200370033003400,4
4,fffe43004900440035003000370031003900,5


In [81]:
print(df_to_submit["churn_risk_score"].value_counts())
print("-"*50)

4    5874
3    5605
5    5078
1    1975
2    1387
Name: churn_risk_score, dtype: int64
--------------------------------------------------


In [82]:
df_to_submit.to_csv(path_or_buf="./df_to_submit"+".csv",float_format='%.0f',index=False)